In [ ]:
import warnings
warnings.filterwarnings("ignore")

from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
!pip freeze

In [ ]:
!pip install seaborn

In [ ]:
show tables;

In [ ]:
diamonds_df = session.table("DIAMONDS")
diamonds_df.show()

In [ ]:
select current_timestamp();







In [ ]:
df = diamonds_df.to_pandas()

import seaborn as sns

# Create a visualization
sns.histplot(
    data=df,
    x="PRICE"
)

In [ ]:
import time
from snowflake.ml.modeling.xgboost import XGBRegressor

CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "X", "Y", "Z"]
LABEL_COLUMNS = ['PRICE']
diamonds_df = session.table("diamonds")

model = XGBRegressor(max_depth=400, input_cols=NUMERICAL_COLUMNS, label_cols=LABEL_COLUMNS)

t0 = time.time()
model.fit(diamonds_df)

t1 = time.time()

print(f"Fit in {t1-t0} seconds.")

In [ ]:
import time
from snowflake.ml.modeling.xgboost import XGBRegressor

CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "X", "Y", "Z"]
LABEL_COLUMNS = ['PRICE']
diamonds_df = session.table("diamonds")

model = XGBRegressor(max_depth=400, input_cols=NUMERICAL_COLUMNS, label_cols=LABEL_COLUMNS, tree_method="gpu_hist", gpu_id=0)

t0 = time.time()
model.fit(diamonds_df)

t1 = time.time()

print(f"Fit in {t1-t0} seconds with GPU.")

In [ ]:
# utils
import requests

### Get logs depending on type
def fetch_log(log_type):
    file_path = f'/var/log/managedservices/{log_type}/mlrs/logs-mlrs.log'
    with open(file_path, 'r') as file:
        # Read the contents of the file
        file_contents = file.read()
        return file_contents

### Get response text
def fetch_metrics(port):
    metrics_url = f"http://localhost:{port}/metrics"
    response = requests.get(metrics_url)
    return response.text

def list_mlrs_metrics():
    txt = fetch_metrics(11501)
    metrics_name_and_value = {}
    for line in txt.split("\n")[:-1]:
        if not line.startswith("#"):
            tokens = line.split(" ")
            name, value = tokens[0], tokens[1]
            metrics_name_and_value[name] = value
        elif line.startswith("# HELP"):
            tokens = line.split(" ")
    return metrics_name_and_value

In [ ]:
print("train attempt", list_mlrs_metrics()['train_attempts_total'])